In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from corrai.measure import MeasuredDats
from copy import deepcopy

# Time series forecasting

This tutorial guides you through the process of training a Machine Learning (ML) model in order to predict at short term ~6h the electricity consumption of a tertiary building office equipments. 

The tutorial will follow these big steps:
- Load the raw data provided by the Building Energy Monitoring System (BEMS)
- Use the Corrai object <code>MeasureDats</code> to pre-process, clean, and visualize the data
- Train several ML model, select the most appropriate and tune its hyper parameters
- Assess and discuss the model performance
- Wrap the model and the processing pipes for production

# Visualise the data

The data are coming from an electrical energy meter that returns an increasing index in kWh. The timestep is variable between 11 min and 22min. The electric equipments are office equipment, and probably some lab equipments

First lets load the csv file containing a single column of data using Pandas

In [ ]:
data_df = pd.read_csv(Path(os.getcwd()) / "resources/compteur_elec.csv", index_col=0, sep=';')
data_df.index = pd.to_datetime(data_df.index, format='mixed', utc=True)

Lets use the <code>DataFrame</code> <code>info()</code> method to get some insights 

In [ ]:
data_df.info()

It looks that data are available from the 2022-12-09 to the 2023-11-22 (in order to get full days).
It also looks that among the 22992 entries, none are Nan. This is good news

We will use the <code>MeasureDats</code> object to get a visual representation and try to identify the gaps that will be annoying (greater than 3h, where a linear the interpolation may become "dangerous").

_Note : this is a bit of a misuse of MeasureDats. For a proper introduction to the class, see the corresponding tutorial_    

In [ ]:
my_data = MeasuredDats(data_df)

In [ ]:
my_data.plot_gaps(gaps_timestep="3H")

In [ ]:
my_data.get_gaps_description(gaps_timedelta='3H')

- A total of 5 gaps greater than 3h minutes were found.
- The greater gap is 4 days long

This is not bad for nearly a full year of data

We can go a bit further and get the date of the gaps

In [ ]:
from corrai.measure import find_gaps

In [ ]:
find_gaps(data_df, timestep='3h')

Given this results, we will probably trash the following periods:
- __2023-04-07__ : 16h is to long for interpolation
- __2023-05-19 to 2023-05-29__ : we choose to trash a full period. As we will see later, we are going to build sequences to train the ML models. Gaps will introduce errors, so we want to limit there number. It is better to throw away some good days
- __2023-10-28 to 2023-10-30__ : the gap span over these two days

Moreover, the previous figure showed an anomaly with the energy meter returning a negative value. This shall be removed*

# Clean the data

The energy meter data returns a monotonously increasing time series.
However we would like to have the electric energy rate [W] absorbed by the equipments.
We propose the following transformations to process the time series :
- remove negative values
- convert the kWh to Joules
- apply time gradient
- Fill in the small gaps (<=3h) using linear interpolation
- Apply Back filling and Front filling operation to make sure no Nan value are present at the beginning and at the and of the time series
- Resample the timeseries to get a constant 15 min timestep
- The resampling introduced new gaps, when the timestep was greater than 15min. Fill them using linear interpolation again.

To easily generate the pipeline and visualise its effect, we use the <code>common_transformations</code> of the <code>MeasureDats</code>. We define a single transformation process called ... <code>process</code>

In [ ]:
from corrai.measure import Transformer, AGG_METHOD_MAP, AggMethod

In [ ]:
my_data.common_trans = {
    "Process": [
        [Transformer.DROP_THRESHOLD, {'lower': 0}],
        [Transformer.APPLY_EXPRESSION, {"expression": "X * 1000 * 3600"}],
        [Transformer.TIME_GRADIENT, {}],
        [Transformer.INTERPOLATE, {"method": "linear"}],
        [Transformer.BFILL, {}],
        [Transformer.FFILL, {}],
        [Transformer.RESAMPLE, {"rule": "15T", "method": AGG_METHOD_MAP[AggMethod.MEAN]}],
        [Transformer.INTERPOLATE, {"method": "linear"}],
    ]
}

my_data.transformers_list = ["Process"]

Lets see the effect of the pipe !

In [ ]:
my_data.plot()

It really looks good ! We can make the following observation :
- There is a weekly pattern with fewer consumption during the weekends
- There is a daily pattern, the energy rate starts to increase around 6 o clock and drops around 17h
- For an unknown reason, there seem to be a pattern of period ~2h. It could be chiller, for a server local for exemple
- It's not very clear, but the energy rate demand seem to increase slightly during the cold months
- There seem to be a holiday period from the 25 December to the 2nd of january

We can also see the gaps in the data. For example, if you try to zoom in on the second half of the month of May, you should see straight lines corresponding the linear interpolation. Lets remove them.

In [ ]:
# List of full days to drop
days_to_drop = [
    "2023-04-07",
    "2023-05-19",
    "2023-05-20",
    "2023-05-21",
    "2023-05-22",
    "2023-05-23",
    "2023-05-24",
    "2023-05-25",
    "2023-05-26",
    "2023-05-27",
    "2023-05-28",
    "2023-05-29",
    "2023-10-28",
    "2023-10-29",
    "2023-10-30",
]

days_to_drop_dt = pd.to_datetime(days_to_drop, errors='coerce')
data_df = data_df[~data_df.index.strftime('%Y-%m-%d').isin(days_to_drop_dt.strftime('%Y-%m-%d'))]

# Add features to the data set

We could use the time series, split it into sequences 18h for example, and try to predict the next 6 hours based on the twelve previous. 
We tried, and got pretty bad results...
It seems logic as only 12 hours is not enough to capture the frequencies mentioned earlier (period of 24h and 7 days).
We could increase the length of the sequence, but we would reduce the number of available sequence, or create to many overlaps which could cause training problems.

One of the common solutions to help the model learn, consists in adding sine wave at the desired frequency. Here is an example : https://www.tensorflow.org/tutorials/structured_data/time_series
You can see these new time series as additional coordinate for the data points. It will make the measure happening at the same time of the day or at the same day of the week closer.

For practical reason, we will "automate" the addition of these sine waves time series using transformers, and the <code>MeasureDats</code> pipeline

In [ ]:
from corrai.transformers import PdAddSineWave

In [ ]:
preprocess_pipeline = deepcopy(my_data.get_pipeline())
preprocess_pipeline.steps.extend([
    ("Sine_7D", PdAddSineWave(frequency=1 / (7 * 24 * 3600), feature_marker="Sine_7D")),
    ("Sine_24h", PdAddSineWave(frequency=1 / (24 * 3600), feature_marker="Sine_24H")),
    ("Sine_2h", PdAddSineWave(frequency=1 / (2 * 3600), feature_marker="Sine_2H"))
])

In a jupyter Notebook, you can easily get a graphical representation of your pipeline

In [ ]:
preprocess_pipeline

Now let's add a columns to the original DataFrame. The values are 0 when there is holidays, and 
1 when it's a day at work. We do not add this using a transformer in a pipeline, since it depends on the occupant behaviour (or their boss's). It is more a "measure" or at least a schedule that must come from the BEMS system.


In [ ]:
data_df["Holidays"] = np.ones(data_df.shape[0])
data_df.loc["2022-12-24":"2023-01-01", "Holidays"] = 0

Before doing further manipulation, we are going to isolate a part of the dataset (let's say ~10% or ~5 weeks).
This is called the test sample. We will not even look at it during the ML training process.
It will only be used to evaluate the model performance

Although there might be a seasonal effect that it is not taken into account, we choose to select the las 2254 timestep as the test sample. It is better to have a coherent sequence and not random fragments of time series

In [ ]:
train_valid_df = data_df.iloc[:data_df.shape[0] - 2254, :]
test_df = data_df.iloc[2254:, :]

Before moving on to sampling and ML model fitting, we are going to scale the features. ML models don't like large distance between points, it is good practice to scale your data. 
Fortunately <code>PdSkTransformer</code> shall make scaling and unscaling easy.
The scaler is append to the <code>preprocess_pipeline</code> pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler
from corrai.transformers import PdSkTransformer

In [ ]:
preprocess_pipeline.steps.append(
    ('Standard_scaler', PdSkTransformer(StandardScaler()))
)

Lastly we push the training / validation data through the pipeline

In [ ]:
scaled_train_val_df = preprocess_pipeline.fit_transform(train_valid_df)

In [ ]:
scaled_train_val_df.loc[scaled_train_val_df["Holidays"] > 1, "Holidays"] = 1
scaled_train_val_df.loc[scaled_train_val_df["Holidays"] < 0, "Holidays"] = 0

# Time Series Sampling


Our objective is to predict the future. We would like to know the energy rate absorbed by the equipments in the next 6h, based on the last 12h.

To do this, we train Machine learning model using short sequence of the time series.
- Features are the last 12h : energy rate, 2h, 24h, 7D sine waves and the holidays.
- The target will be the next 6h energy rate  

For example, the timeseries :

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

with a <code>sequence_length = 5</code> and <code>sampling_rate = 1</code>  will become:

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
...
[6, 7, 8, 9, 10]

For a 2D time series (columns are features, index is time step), the output will be a 3D numpy array of shape [_batch size, time steps, dimensionality_] where _dimensionality_ is the number of features [Aurélien Géron Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow]

The sampling of the time series is done using the function <code>time_series_sampling</code>

We will use <code>shuffle=True</code> to make sure sequences ... are shuffled, so when we split the dataset between training and validation we get sequences spread across the year, and not a single season

In [ ]:
from corrai.learning.model_selection import time_series_sampling

In [ ]:
train_valid_sequences = time_series_sampling(
    preprocess_pipeline.fit_transform(train_valid_df),
    sequence_length=4 * (12 + 6), #12h history + 6h in the future at a 15min timmstep,
    sampling_rate=1,
    shuffle=True,
    seed=42 # Make sure the behaviour can be repeated in the notebook
)

In [ ]:
train_valid_sequences.shape

The shape of the obtained array is [29993, 72, 5] corresponding to the number of sequences, the number of time steps and the number of features.

The number of sequences is elevated, but we used <code>sampling_rate=1</code>, meaning that there will be a lot of overlaping between the sequences. This may lead to model over fitting, or artificially improve evaluation metrics, as part of the training sample will also be present in validation sample.

That is why final performance will be evaluated on the test sample

It is now time to isolate the target from the sequences. 
Using the previous example :
[0, 1, 2, 3, 4] 
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
 … 
[6, 7, 8, 9, 10]

With an history of <code>n_step_history=3</code> and <code>n_step_future=2</code> we get
X = [0, 1, 2] 
    [1, 2, 3]
    [2, 3, 4]
     … 
    [6, 7, 8]
    
Y = [3, 4] 
    [4, 5]
    [5, 6]
     … 
    [9, 10]

We also want to split X and Y into two pair of samples X_train, y_train, X_valid, y_valid

To do this we will use the <code>sequences_train_test_split</code> function


In [ ]:
from corrai.learning.model_selection import sequences_train_test_split

In [ ]:
x_train, x_valid, y_train, y_valid = sequences_train_test_split(
    train_valid_sequences,
    targets_index=0,
    n_steps_history=12 * 4,
    n_steps_future=6 * 4,
    train_size=0.80,
    shuffle=False
)

# Model testing

In [ ]:
from corrai.learning.time_series import TsLinearModel, TsDeepNN

In [ ]:
ts_linear = TsLinearModel()

In [ ]:
ts_linear.fit(x_train, y_train, x_valid, y_valid)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print(f"R² = {ts_linear.score(x_valid, y_valid)} \n"
      f"MSE = {mean_squared_error(ts_linear.predict(x_valid), y_valid)}")

In [ ]:
from corrai.learning.time_series import plot_sequence_forcast

In [ ]:
plot_sequence_forcast(x_valid, y_valid, model=ts_linear, batch_nb=32)

## Deep Neural Network

In [ ]:
deep_rnn = TsDeepNN()

In [ ]:
deep_rnn.fit(x_train, y_train, x_valid, y_valid)

In [ ]:
print(f"R² = {deep_rnn.score(x_valid, y_valid)} \n"
      f"MSE = {mean_squared_error(deep_rnn.predict(x_valid), y_valid)}")

In [ ]:
plot_sequence_forcast(x_valid, y_valid, model=deep_rnn, batch_nb=32)